In [26]:
import numpy as np
import pandas as pd
from scipy.stats import distributions as iid
from matplotlib import pyplot as plt
import statsmodels.api as sm

In [27]:
data = pd.read_stata('/Users/cassturk/Downloads/angrist-krueger91.dta')

In [28]:
#adding covariates race, smsa, married, region-of-residence

j =pd.get_dummies(data['yob'])

m = data['yob'].astype(str)+data['qob'].astype(str)
l = pd.get_dummies(m)

n = pd.get_dummies(data['region'])

d = np.array([list(data['logwage'])])
a= np.array([list(data['edu'])])


#print(np.array(data['black']).T.shape)

b = np.r_[a,np.array(j).T, np.array([list(data['black'])]), np.array([list(data['smsa'])]), np.array([list(data['married'])]) ,np.array(n).T]

beta_ols = np.linalg.solve(b@b.T, b@d.T)

#note it's pretty good:
print(beta_ols)

[[ 0.06324573]
 [-3.97086325]
 [-3.97490852]
 [-3.97932498]
 [-3.97920826]
 [-3.98201632]
 [-3.99168176]
 [-3.99124945]
 [-3.9938987 ]
 [-3.99303127]
 [-4.00149961]
 [-0.2574833 ]
 [ 0.17630067]
 [ 0.24786744]
 [ 8.81099095]
 [ 8.82696207]
 [ 8.64654158]
 [ 8.75822556]
 [ 8.71889754]
 [ 8.69760706]
 [ 8.67168177]
 [ 8.70321843]
 [ 8.7078125 ]]


In [31]:
data1 = pd.read_stata('/Users/cassturk/Downloads/angrist-krueger91.dta')

oo = 0
for i in range(20):
    #my computer can't compute the whole projection matrix so here's some sampling lol
    data = data1.sample(n=10000)
    
    #creating dummies
    j =pd.get_dummies(data['yob'])
    k =pd.get_dummies(data['qob'])
    n = pd.get_dummies(data['region'])

    m = data['yob'].astype(str)+data['qob'].astype(str)

    l = pd.get_dummies(m)

    #dropping linear combinations

    l = l.drop(columns=['19393', '19391', '19392', '19394', '19303', '19313', '19323', 
                        '19333', '19343','19353', '19363','19373','19383'])

    j = j.drop(columns = [1939])

    k = k.drop(columns = [3])


    #creating matrices
    y = np.array([list(data['logwage'])]).squeeze()
    a = np.array([list(data['edu'])])

    x = np.r_[np.array([list(data['edu'])]), np.array(j).T, np.array([list(data['black'])]), 
              np.array([list(data['smsa'])]), np.array([list(data['married'])]), np.array(n).T].squeeze()
    
    

    #computation heavy stuff
    z = np.r_[np.array(l).T, np.array(j).T, np.array([list(data['black'])]), 
              np.array([list(data['smsa'])]), np.array([list(data['married'])]), np.array(n).T]
    z = z.astype('float32')
    zint = np.linalg.pinv(z@z.T)
    
    #checking with canned functions
    model = sm.OLS(a.T,z.T)
    results = model.fit()
    reses = results.params
    
    
    xhat = reses@z
    
    xs = np.r_[[xhat.T], np.array(j).T, np.array([list(data['black'])]), 
              np.array([list(data['smsa'])]), np.array([list(data['married'])]), np.array(n).T]
    model = sm.OLS(y.T,xs.T)
    results = model.fit()
    print(results.params[0])
    #canned gives same answers as both paper and samples

    #the really computation-heavy stuff, according to where my computer chooses to die most frequently
    zpr = z.T@zint
    zproj = zpr@z
    xzpr =x@zproj
    xzprx = xzpr@x.T
    zpry = xzpr@y.T
    beta_iv = np.linalg.solve(xzprx, zpry)
    beta_iv_x = beta_iv.squeeze()
    oo= oo+beta_iv_x[0]
    print(beta_iv_x[0])
print('average')
print(oo/20)

0.11814703646822465
0.11814718183614766
0.10123569417824763
0.10123566894347609
0.0962617068148412
0.09626172797990562
0.07536454183579794
0.07536461631531334
0.12819447513911936
0.12819449897962326
0.10495157068236624
0.10495172051619787
0.04451915993167836
0.044519132805586926
0.03804087318405239
0.03804087652504282
0.1984588274548625
0.19845865991091355
0.11534404601641099
0.11534417915445751
0.023217065169580164
0.02321716248871706
0.10824750308874667
0.10824755199841421
0.04463249490390031
0.04463246386803201
0.03121839019473477
0.03121841226335991
0.0952817053169015
0.0952818149416713
0.007299727865696715
0.007299762036271265
0.07375767042105874
0.07375761556790608
0.04494204878831433
0.0449419986088315
0.02270367212072244
0.022703654473896495
0.024431399472088153
0.024431385053995225
average
0.0007481250421338799
